# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 0: Preparation and import data from s3
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [5]:
#Before we continue, we need to install related python package.
import sys

!{sys.executable} -m pip install boto3
!{sys.executable} -m pip install s3fs
!{sys.executable} -m pip install pyspark
!{sys.executable} -m pip install --upgrade pandas
!{sys.executable} -m pip install cqlsh

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already up-to-date: pandas in /Users/ishirunkang/anaconda3/lib/python3.6/site-packages (1.0.5)
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
# Do all imports and installs here
import configparser
import pandas as pd
import os
import boto3
import uuid

In [7]:
config = configparser.ConfigParser()
config.read('iam.cfg')
os.environ['AWS_ACCESS_KEY_ID']=config['AWS_CREDS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS_CREDS']['AWS_SECRET_ACCESS_KEY']

client=boto3.client('s3')

global_land_temperature_url = 's3://srk-data-eng-capstone/GlobalLandTemperaturesByCity.csv'
airport_codes_url = 's3://srk-data-eng-capstone/airport-codes_csv.csv'
us_city_demographics_url = 's3://srk-data-eng-capstone/us-cities-demographics.csv'


In [26]:
# globalLandTemperature = pd.read_csv(global_land_temperature_url)
# globalLandTemperature.head(1)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E


### Scope the Project and Gather Data

#### Project description:

This project will be separate to multiple parts, and all four dataset will be used. 

Before we talk about the details, we need to know the characteristics of relational DB and non-relational DB.

For relational DB, its characteristics is low redundancy and high completeness, which means it is very suitable for small or medium size data, and the database does not change so much. In our case, we should store temperature, airport code and US cities demographic data into a relational database that meets 3NF because it does not always change so much and the volume of data is not that large.

For non-relational DB, its characteristics is higher elasticity, faster read & write speed and evoving data volume. In our case, we should save I94 data into non-relational DB. Because this piece of data need to make ETL process almost every minutes in real world background, and it need dynamic write and read for real-time data monitoring.


The final solution will work as a a database management system. When user input the time or time period and the column they interested in (e.g., visa type), the system will return related data as a dataframe. For example, a user needs to know where are the busiest airport for investment visa holder(E-1 visa) in 2016 and its basic information such as temperature, and the status of the city such as population(age, majority race, etc.), or when is the peak-time for international student come to the United States and where are they come from.

* Data will be imported from Amazon S3
* Relational DB will be implement on AWS Redshift
* Non-Relationalship DB will be implement on Amazon Keyspace
* Data cleaning and ETL process will be implement on Amazon EMR with Spark

#### The dataset is going to use in this project are:

* I94 Immigration Data: This data comes from the US National Tourism and Trade Office. A data dictionary is included in the workspace. https://travel.trade.gov/research/reports/i94/historical/2016.html is where the data comes from. There's a sample file so you can take a look at the data in csv format before reading it all in. You do not have to use the entire dataset, just use what you need to accomplish the goal you set at the beginning of the project.
* World Temperature Data: This dataset came from Kaggle. You can read more about it here: https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data.
* U.S. City Demographic Data: This data comes from OpenSoft. You can read more about it here: https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/.
* Airport Code Table: This is a simple table of airport codes and corresponding cities. It comes from here:https://datahub.io/core/airport-codes#data.

In [16]:
	
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

i94_url='s3a://srk-data-eng-capstone/i94/i94_jan16_sub.sas7bdat'
i94 = pd.read_sas(i94_url, 'sas7bdat',encoding="ISO-8859-1").drop_duplicates()
i94['id_'] = pd.Series([uuid.uuid1() for each in range(len(i94))])

sql = '''SELECT id_, cicid, i94yr, i94mon, i94cit, i94res, i94port,
       arrdate, i94mode, i94addr, depdate, i94bir, i94visa,
       count, dtadfile, visapost, occup, entdepa, entdepd,
       entdepu, matflag, biryear, dtaddto, gender, insnum,
       airline, admnum, fltno, visatype FROM i94'''

while True:
    if os.path.isdir('16_jan.parquet'):
        break
    else:
        i94.to_csv('16_jan.csv')
        df_spark=spark.read.option('header','true').csv('16_jan.csv')
        df_spark.createOrReplaceTempView('i94')
        parquet_data=spark.sql(sql)
        parquet_data.write.parquet('16_jan.parquet',mode='overwrite')
        os.system('aws s3 cp 16_jan.parquet s3://i94-backup --recursive')
        break        




In [9]:
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1, CERT_REQUIRED
from cassandra.auth import PlainTextAuthProvider


ssl_context = SSLContext(PROTOCOL_TLSv1)
ssl_context.load_verify_locations('AmazonRootCA1.pem')
ssl_context.verify_mode = CERT_REQUIRED
auth_provider = PlainTextAuthProvider(username=str(config['APACHE_CASSANDRA_CREDS']['CASSANDRA_USERNAME']), password=str(config['APACHE_CASSANDRA_CREDS']['CASSANDRA_PASSWORD']))
cluster = Cluster(['cassandra.eu-west-1.amazonaws.com'], ssl_context=ssl_context, auth_provider=auth_provider, port=9142)

In [24]:
session = cluster.connect()
create_keyspace="""CREATE KEYSPACE IF NOT EXISTS "i94"
                   WITH REPLICATION={'class':'SingleRegionStrategy'}"""
session.execute(create_keyspace)
test=session.execute('SELECT * FROM system_schema.keyspaces')
print(test.current_rows)

[Row(keyspace_name='system_schema', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'com.amazonaws.cassandra.DefaultReplication')])), Row(keyspace_name='system_schema_mcs', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'com.amazonaws.cassandra.DefaultReplication')])), Row(keyspace_name='system', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'com.amazonaws.cassandra.DefaultReplication')])), Row(keyspace_name='i94', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'com.amazonaws.cassandra.DefaultReplication')]))]


In [25]:
create_table="""CREATE TABLE IF NOT EXISTS "i94".i94 (
                                                      cicid DOUBLE,
                                                      i94yr DOUBLE,
                                                      i94mon DOUBLE,
                                                      i94cit DOUBLE,
                                                      i94res DOUBLE,
                                                      i94port TEXT,
                                                      arrdate DOUBLE,
                                                      i94mode DOUBLE,
                                                      i94addr DOUBLE,
                                                      depdate TEXT,
                                                      i94bir DOUBLE,
                                                      i94visa DOUBLE,
                                                      count DOUBLE,
                                                      dtadfile DOUBLE,
                                                      visapost TEXT,
                                                      occup TEXT,
                                                      entdepa TEXT,
                                                      entdepd TEXT,
                                                      entdepu TEXT,
                                                      matflag TEXT,
                                                      biryear DOUBLE,
                                                      dtaddto TEXT,
                                                      gender TEXT,
                                                      insnum TEXT,
                                                      airline TEXT,
                                                      admnum DOUBLE,
                                                      fltno TEXT,
                                                      visatype TEXT,
                                                      id_ TEXT,
                                                      PRIMARY KEY(id_)
                ) """
session.execute(create_table)
test_two=session.execute("SELECT * from system_schema.tables WHERE keyspace_name='i94';")
print(test_two.current_rows)

[Row(keyspace_name='i94', table_name='i94', bloom_filter_fp_chance=0.01, caching=OrderedMapSerializedKey([('class', 'com.amazonaws.cassandra.DefaultCaching')]), cdc=None, comment='', compaction=OrderedMapSerializedKey([('class', 'com.amazonaws.cassandra.DefaultCompaction')]), compression=OrderedMapSerializedKey([('class', 'com.amazonaws.cassandra.DefaultCompression')]), crc_check_chance=1.0, dclocal_read_repair_chance=0.0, default_time_to_live=0, extensions=OrderedMapSerializedKey([]), flags=SortedSet(['compound']), gc_grace_seconds=7776000, id=UUID('eb5bb07a-8110-391c-b125-8fd997124550'), max_index_interval=2048, memtable_flush_period_in_ms=3600000, min_index_interval=128, read_repair_chance=0.0, speculative_retry='99PERCENTILE')]


In [29]:
test_three=session.execute("SELECT * FROM system_schema.columns WHERE keyspace_name = 'i94' AND table_name = 'i94';")
print(test_three.current_rows)

[Row(keyspace_name='i94', table_name='i94', column_name='id_', clustering_order='none', column_name_bytes=b'id_', kind='partition_key', position=0, type='text'), Row(keyspace_name='i94', table_name='i94', column_name='admnum', clustering_order='none', column_name_bytes=b'admnum', kind='regular', position=-1, type='double'), Row(keyspace_name='i94', table_name='i94', column_name='airline', clustering_order='none', column_name_bytes=b'airline', kind='regular', position=-1, type='text'), Row(keyspace_name='i94', table_name='i94', column_name='arrdate', clustering_order='none', column_name_bytes=b'arrdate', kind='regular', position=-1, type='double'), Row(keyspace_name='i94', table_name='i94', column_name='biryear', clustering_order='none', column_name_bytes=b'biryear', kind='regular', position=-1, type='double'), Row(keyspace_name='i94', table_name='i94', column_name='cicid', clustering_order='none', column_name_bytes=b'cicid', kind='regular', position=-1, type='double'), Row(keyspace_nam

In [ ]:
while True:
    try:
        os.remove('16_jan.csv')
    except:
        break

In [19]:
data=spark.read.parquet('16_jan.parquet')
data.createOrReplaceTempView('temp')
result=spark.sql('SELECT * FROM temp LIMIT 5')
result.show()

+--------------------+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-----------+-----+--------+
|                 id_|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|     admnum|fltno|visatype|
+--------------------+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-----------+-----+--------+
|ad9a5e74-c6a5-11e...|  7.0|2016.0|   1.0| 101.0| 101.0|    BOS|20465.0|    1.0|     MA|   null|  20.0|    3.0|  1.0|    null|    null| null|      T|   null|   null|   null| 1996.0|     D/S|     M|  null|     LH|346608285.0|  424|      F1|
|ad9a5fa8-c6a5-11e...|  8.0|2016.0|   1.

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.